In [ ]:
# asume que los datos estan en ../data/<año>.csv.bz2
# fixme: poner el directorio de data que esté en el repo cuando pusheen los pibes

import seaborn as sns
import matplotlib.pyplot as plt

import fnmatch

import pandas as pd
import time
import os
import shutil
#sns.set(color_codes=True)

# load data
tmp_dir = "./tmp_csv/"
#years = ["2006", "2007"]
years = ["2007"]

## Procesar datos

In [ ]:
#crear directorio con data de delays agrupada por retraso 
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
    
for year in years:
    if not os.path.exists(tmp_dir+year+".csv"):
        print("Leyendo csv: {}".format(year))
        t = time.time()
        df = pd.read_csv("../data/"+year+".csv.bz2", compression="bz2", usecols=["Month", "Year", "DayofMonth", "DayOfWeek", "DepDelay", "ArrDelay"])    

        #formato mas feliz para fechas
        dates = pd.to_datetime(df.Year*10000+df.Month*100+df.DayofMonth,format='%Y%m%d')
        df["date"] = dates
        df = df[["date", "DepDelay", "ArrDelay"]]

        df = df.groupby(by=df['date'].dt.date).mean()
        df.to_csv(tmp_dir+year+".csv")
        del df 
    

In [ ]:
for filename in os.listdir(tmp_dir):
    if filename.endswith(".csv") and filename != "merged.csv": 
        print(filename)
        df = pd.read_csv(tmp_dir+filename)
        fig, ax = plt.subplots(figsize=(16,4))
        sns.lineplot(x=df.date, y=df.DepDelay, linestyle='-')
        plt.xticks(plt.xticks()[0], df.date, rotation=90)
        plt.tight_layout()
        xticks = ax.xaxis.get_major_ticks()
        for i in range(len(xticks)):
            if i%30 != 0:
                xticks[i].set_visible(False)
            if i%int((len(xticks))/4) == 0:    
                plt.axvline(x=i, color="orange")
            if i%int((len(xticks))/3) == 0:    
                plt.axvline(x=i, color="green")
        fig.autofmt_xdate()
        plt.show()
        
#Trimestres en verde, cuatrimestres en naranja

In [ ]:
#merge
df = pd.DataFrame(columns=["date", "DepDelay", "ArrDelay"])
for filename in os.listdir(tmp_dir):
    if filename.endswith(".csv") and filename != "merged.csv":
        print(filename)
        df_input = pd.read_csv(tmp_dir+filename)
        df = pd.concat([df, df_input], sort=False)
df.to_csv(tmp_dir+"merged.csv")

In [ ]:
df = pd.read_csv(tmp_dir+"merged.csv")
df.plot(x="date", y="DepDelay", linestyle='-', figsize=(16,4))

## CML acá 

In [ ]:
delays = df["DepDelay"]
print(delays)
len(delays)

In [ ]:
import numpy as np
import math

n = 5
phases = [i*math.pi/2 for i in range(4)]

# semanas, meses, bimestres, trimestres, cuatrimestres, semestres, años
freqs =  [7] + [355/i for i in [12, 6, 4, 3, 2, 1]]

def trig_vals(t):
    return  [math.sin(2 * math.pi / f * t + p) for f in freqs for p in phases] + \
            [math.cos(2 * math.pi / f * t + p) for f in freqs for p in phases]

A = np.array([])
arrays = []
for i in range(len(delays)):
    row = np.array([i**p for p in range(n+1)] + trig_vals(i))
    arrays.append(row)

A = np.stack(arrays)
print(A.shape)

coefs = np.linalg.lstsq(A, delays, rcond=None)[0]

In [ ]:

from cml import svd_solver as ss
coefs_svd = ss.lstsq(A, delays)

from cml import normal_equations_solver as nes
coefs_nes = nes.lstsq(A, delays)

from cml import qr_solver as qr_fast
coefs_qr_fast = qr_fast.lstsq(A, delays)
from cml import qr_col_pivot_solver as qr_col
coefs_qr_col = qr_col.lstsq(A, delays)
from cml import qr_full_pivot_solver as qr_full
coefs_qr_full = qr_full.lstsq(A, delays)


In [ ]:
df["dl_pred"] = A@coefs
df["dl_pred_nes"] = A@coefs_nes
df["dl_pred_svd"] = A@coefs_svd

df["dl_pred_qr"] = A@coefs_qr_fast
df["dl_pred_qr_col"] = A@coefs_qr_col
df["dl_pred_qr_full"] = A@coefs_qr_full

In [ ]:
def plot_impl(impl_df, impl_label):

    plt.clf()

    plt.figure(figsize=(16,5))
    plt.xlabel('Predicción para el i-ésimo dia')

    ax1 = df.DepDelay.plot(color='blue', grid=True, secondary_y=True, label='real delay')
    ax2 = df.dl_pred.plot(color='orange', grid=True, label='pred delay (np)')

    ax3 = df[impl_df].plot(color='magenta', grid=True, label=impl_label)

    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()

    plt.legend(h1+h2, l1+l2, loc=2)
    plt.show()

plot_impl("dl_pred_nes", "pred delay (normal equations)")
plot_impl("dl_pred_svd", "pred delay (SVD)")
plot_impl("dl_pred_qr", "pred delay (QR)")
plot_impl("dl_pred_qr_col", "pred delay (QR Col Pivot)")
plot_impl("dl_pred_qr_full", "pred delay (QR Full Pivot)")


#ax6 = df.dl_pred_qr_col.plot(color=(0.3, 0.5, 0.1), grid=True, label='pred delay (qr_col)')
#ax7 = df.dl_pred_qr_full.plot(color=(0.1, 1.0, 0.2), grid=True, label='pred delay (qr_full)')

In [ ]:
#para jugar con fases
time        = np.arange(0, 12,0.1);
amplitude   = np.sin(2 * math.pi / 1* time + math.pi)
plt.plot(time, amplitude)
plt.show()

## BORRAR ARCHIVOS

In [ ]:
#borrar aux de años
for filename in os.listdir(tmp_dir):
    if fnmatch.fnmatch(filename, '*.csv') and filename != "merged.csv":
        os.remove(tmp_dir+filename)

In [ ]:
#borrar tmp
shutil.rmtree(tmp_dir)